In [1]:
import numpy as np
import pandas as pd
import pyarrow.feather as feather
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

df = feather.read_feather('data/artsy_full.feather')

In [2]:
df.permalink[0]

'https://www.artsy.net/artwork/gustav-klimt-der-kuss-the-kiss'

In [3]:
import loginkey

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

from urllib.parse import unquote

import re

import time as time
import json

In [7]:
def initialize_driver():
    
    from selenium.webdriver.chrome.options import Options
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    
    print('initializing driver...')
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://www.artsy.net/')
    
    print('loaded artsy.net, waiting to log in...')
    
    wait = WebDriverWait(driver, 10)
    time.sleep(2)

    button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div/header/div/nav/div[1]/a[2]')
    driver.execute_script("arguments[0].click();", button)

    user = loginkey.user
    password = loginkey.password
    
    time.sleep(2)
    
    print('logging in...')
    
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hgOWHr')))

    driver.find_element(By.CLASS_NAME, 'hgOWHr').click()
    driver.find_element(By.XPATH, "//input[@name='email']").send_keys(user)
    driver.find_element(By.CLASS_NAME, 'jGXstL').click()
    driver.find_element(By.XPATH, "//input[@name='password']").send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    
    print('log in sucessful...')
    
    return driver

In [12]:
def get_auction_price(row):
    try:
        driver.get(row.permalink)

        soup = BeautifulSoup(driver.page_source)

        json_script = soup.find_all('script')[-7]
        json_script = json_script.decode_contents().replace('\n              var __RELAY_BOOTSTRAP__ = ','').replace(';\n            ','')
        json_string = json.loads(json_script)
        json_dict = json.loads(json_string)[0][1]

        try:
            price = json_dict['json']['data']['artwork']['artists'][0]['auctionResultsConnection']['edges'][0]['node']['price_realized']['display']
        except:
            price = float('nan')
    except:
        price = float('nan')
        
    row['price'] = price
    
    return row

In [ ]:
driver = initialize_driver()
time.sleep(5)
df_w_prices = df[:20].progress_apply(get_auction_price, axis=1)

initializing driver...
loaded artsy.net, waiting to log in...
logging in...
log in sucessful...


 80%|██████████████████████████████████▍        | 16/20 [00:38<00:09,  2.42s/it]

In [ ]:
df_w_prices.price